In [213]:
import cloudscraper
from bs4 import BeautifulSoup
import pandas as pd
scraper = cloudscraper.create_scraper()  # returns a requests-like object
base_url = 'https://www.basketball-reference.com/leagues/NBA_{}_per_game.html'
base_url2 = 'https://www.basketball-reference.com/leagues/NBA_{}_advanced.html'

In [214]:
res1 = scraper.get(base_url.format(2025))
soup1 = BeautifulSoup(res1.text, 'html.parser')
table1 = soup1.find_all('table')[0]

    

In [215]:

headers = table1.find_all('th')[1:30]


In [216]:
res2 = scraper.get(base_url2.format(2025))
soup2 = BeautifulSoup(res2.text, 'html.parser')
table2 = soup2.find_all('table')[0]


In [217]:

headers2 = table2.find_all('th')[1:28]

In [218]:
header_table2 = [header.text for header in headers2]
header_table2.insert(1, 'Season')
header_table2.insert(2, 'id')

In [219]:
header_table = [header.text for header in headers]
header_table.insert(1, 'Season')
header_table.insert(2, 'id')


In [220]:



years = range(2020,2026)

def get_data(link, headers, slice):
    df = pd.DataFrame(columns=headers)
    for year in years:

        url = link.format(year)
        res = scraper.get(url)
        soup = BeautifulSoup(res.text, 'html.parser')
        table = soup.find_all('table')[0]
        rows = table.find_all('tr')
    
        for row in rows[1:736]:
            row_data = row.find_all('td')
            individual_data = [data.text for data in row_data[slice]]
            a = row.find('a')
            player_id = ''
            
            
            if a and '/players/' in a.get('href', ''):
                href = a['href']
                player_id = href[11:-5]
            
            individual_data.insert(1, str(year)) 
            individual_data.insert(2, player_id) 
            
          
            df.loc[len(df)] = individual_data            
    return df
    

In [221]:
df = get_data(base_url, header_table, slice(0,29))


In [222]:

df2 = get_data(base_url2, header_table2, slice(0,27))
df2



,Player,Season,id,Age,Team,Pos,G,GS,MP,PER,...,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP
0,CJ McCollum,2020,mccolcj01,28,POR,SG,70,70,2556,17.0,...,8.1,25.9,3.5,0.6,4.1,.077,2.3,-1.8,0.5,1.6
1,Devin Booker,2020,bookede01,23,PHO,SG,70,70,2512,20.6,...,14.9,30.0,6.0,1.5,7.5,.143,3.4,-1.3,2.2,2.6
2,James Harden,2020,hardeja01,30,HOU,SG,68,68,2483,29.1,...,14.2,36.3,9.9,3.2,13.1,.254,8.1,1.6,9.6,7.3
3,Harrison Barnes,2020,barneha02,27,SAC,PF,72,72,2482,13.3,...,8.8,17.4,3.4,1.1,4.6,.088,0.0,-1.3,-1.3,0.5
4,Damian Lillard,2020,lillada01,29,POR,PG,66,66,2474,26.9,...,11.0,30.3,10.9,0.7,11.6,.225,8.3,-0.9,7.5,5.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4238,Jesse Edwards,2025,edwarje01,24,MIN,C,2,0,5,-0.1,...,,0.0,0.0,0.0,0.0,.111,-8.8,4.7,-4.0,0.0
4239,James Wiseman,2025,wisemja01,23,IND,C,1,0,5,31.2,...,0.0,42.2,0.0,0.0,0.0,.237,6.0,-4.4,1.6,0.0
4240,Alondes Williams,2025,willial06,25,DET,SG,1,0,4,53.4,...,0.0,21.1,0.0,0.0,0.0,.565,37.9,6.2,44.1,0.0
4241,Zyon Pullin,2025,pullizy01,23,MEM,SG,3,0,3,-11.7,...,0.0,13.4,0.0,0.0,0.0,-0.304,-15.0,-7.7,-22.7,0.0


In [223]:
df['key'] = df['Player'] + '_' + df['Season']
df2['key'] = df2['Player'] + '_' + df2['Season']


In [224]:
df2 = df2.drop(columns=['Player', 'Season', 'Age', 'Team', 'Pos', 'G', 'GS', 'MP','id'])

In [225]:
df2

,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,...,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,key
0,17.0,.541,.378,.136,1.9,10.2,6.1,19.1,1.0,1.4,...,25.9,3.5,0.6,4.1,.077,2.3,-1.8,0.5,1.6,CJ McCollum_2020
1,20.6,.618,.310,.397,1.3,12.0,6.6,30.0,0.9,0.6,...,30.0,6.0,1.5,7.5,.143,3.4,-1.3,2.2,2.6,Devin Booker_2020
2,29.1,.626,.557,.528,2.9,16.0,9.4,35.9,2.3,2.1,...,36.3,9.9,3.2,13.1,.254,8.1,1.6,9.6,7.3,James Harden_2020
3,13.3,.574,.338,.337,3.4,12.9,8.1,9.2,0.8,0.5,...,17.4,3.4,1.1,4.6,.088,0.0,-1.3,-1.3,0.5,Harrison Barnes_2020
4,26.9,.627,.500,.384,1.4,10.5,6.0,34.4,1.3,0.8,...,30.3,10.9,0.7,11.6,.225,8.3,-0.9,7.5,5.9,Damian Lillard_2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4238,-0.1,,,,0.0,0.0,0.0,23.6,0.0,0.0,...,0.0,0.0,0.0,0.0,.111,-8.8,4.7,-4.0,0.0,Jesse Edwards_2025
4239,31.2,.615,.250,.500,0.0,22.1,11.1,0.0,0.0,0.0,...,42.2,0.0,0.0,0.0,.237,6.0,-4.4,1.6,0.0,James Wiseman_2025
4240,53.4,1.250,.500,.000,0.0,0.0,0.0,65.0,0.0,0.0,...,21.1,0.0,0.0,0.0,.565,37.9,6.2,44.1,0.0,Alondes Williams_2025
4241,-11.7,.000,.000,.000,0.0,0.0,0.0,0.0,0.0,0.0,...,13.4,0.0,0.0,0.0,-0.304,-15.0,-7.7,-22.7,0.0,Zyon Pullin_2025


In [226]:
combined = pd.merge(df, df2, on='key')
combined

,Player,Season,id,Age,Team,Pos,G,GS,MP,FG,...,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP
0,James Harden,2020,hardeja01,30,HOU,SG,68,68,36.5,9.9,...,14.2,36.3,9.9,3.2,13.1,.254,8.1,1.6,9.6,7.3
1,Bradley Beal,2020,bealbr01,26,WAS,SG,57,57,36.0,10.4,...,11.4,34.4,4.5,0.6,5.1,.118,5.3,-2.4,2.9,2.5
2,Damian Lillard,2020,lillada01,29,POR,PG,66,66,37.5,9.5,...,11.0,30.3,10.9,0.7,11.6,.225,8.3,-0.9,7.5,5.9
3,Trae Young,2020,youngtr01,21,ATL,PG,60,60,35.3,9.1,...,16.2,34.9,5.3,0.6,5.9,.133,6.2,-2.3,3.9,3.1
4,Giannis Antetokounmpo,2020,antetgi01,25,MIL,PF,63,63,30.4,10.9,...,13.2,37.5,6.1,5.0,11.1,.279,7.4,4.1,11.5,6.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7148,Riley Minix,2025,minixri01,24,SAS,SF,1,0,7.0,0.0,...,0.0,6.1,0.0,0.0,0.0,-0.114,-8.5,-3.3,-11.8,0.0
7149,Jahlil Okafor,2025,okafoja01,29,IND,C,1,0,3.0,0.0,...,,0.0,0.0,0.0,0.0,.196,2.5,4.9,7.5,0.0
7150,Zyon Pullin,2025,pullizy01,23,MEM,SG,3,0,1.0,0.0,...,0.0,13.4,0.0,0.0,0.0,-0.304,-15.0,-7.7,-22.7,0.0
7151,Isaiah Stevens,2025,steveis01,24,MIA,PG,3,0,2.0,0.0,...,0.0,14.8,0.0,0.0,0.0,-0.196,-19.5,7.7,-11.8,0.0


In [227]:
combined = combined[combined['Player'] != 'League Average']


In [228]:
combined.to_csv(r'C:\Users\LL205\Documents\Machine Learning\NBA_predictions\Nba_stats.csv')